In [10]:
import argparse
import glob
import math
import ntpath
import os
import shutil
import urllib
import urllib2

from datetime import datetime

import numpy as np
import pandas as pd

from mne import Epochs, pick_types, find_events
from mne.io import concatenate_raws, read_raw_edf

import dhedfreader

In [11]:
# Label values
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4
UNKNOWN = 5

stage_dict = {
    "W": W,
    "N1": N1,
    "N2": N2,
    "N3": N3,
    "REM": REM,
    "UNKNOWN": UNKNOWN
}

class_dict = {
    0: "W",
    1: "N1",
    2: "N2",
    3: "N3",
    4: "REM",
    5: "UNKNOWN"
}

ann2label = {
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4,
    "Sleep stage ?": 5,
    "Movement time": 5
}

EPOCH_SEC_SIZE = 30

In [15]:
data_dir = 'data/'
output_dir = 'data/output/'
select_ch = ['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'Resp oro-nasal', 'EMG submental', 'Temp rectal']

# Read raw and annotation EDF files
psg_fnames = glob.glob(os.path.join('data/', "*PSG.edf"))
ann_fnames = glob.glob(os.path.join('data/', "*Hypnogram.edf"))
psg_fnames.sort()
ann_fnames.sort()
psg_fnames = np.asarray(psg_fnames)
ann_fnames = np.asarray(ann_fnames)

for i in range(len(psg_fnames)):
    # if not "ST7171J0-PSG.edf" in psg_fnames[i]:
    #     continue

    raw = read_raw_edf(psg_fnames[i])
    sampling_rate = raw.info['sfreq']

    raw = read_raw_edf(psg_fnames[i])
    sampling_rate = raw.info['sfreq']

    raw_eeg_fpz = raw.to_data_frame(scale_time=100.0)['EEG Fpz-Cz']
    raw_eeg_fpz = raw_eeg_fpz.to_frame()
    raw_eeg_fpz.set_index(np.arange(len(raw_eeg_fpz)))
    print('-------------------------------------------------------------------')
    raw_eeg_pz = raw.to_data_frame(scale_time=100.0)['EEG Pz-Oz']
    raw_eeg_pz = raw_eeg_pz.to_frame()
    raw_eeg_pz.set_index(np.arange(len(raw_eeg_pz)))
    print('-------------------------------------------------------------------')

    raw_eog = raw.to_data_frame(scale_time=100.0)['EOG horizontal']
    raw_eog = raw_eog.to_frame()
    raw_eog.set_index(np.arange(len(raw_eog)))
    print('-------------------------------------------------------------------')

    raw_resp = raw.to_data_frame(scale_time=100.0)['Resp oro-nasal']
    raw_resp = raw_resp.to_frame()
    raw_resp.set_index(np.arange(len(raw_resp)))
    print('-------------------------------------------------------------------')

    raw_emg = raw.to_data_frame(scale_time=100.0)['EMG submental']
    raw_emg = raw_emg.to_frame()
    raw_emg.set_index(np.arange(len(raw_emg)))
    print('-------------------------------------------------------------------')

    raw_temp = raw.to_data_frame(scale_time=100.0)['Temp rectal']
    raw_temp = raw_temp.to_frame()
    raw_temp.set_index(np.arange(len(raw_temp)))
    print('-------------------------------------------------------------------')

    raw_ch_df = pd.concat([raw_eeg_fpz, raw_eeg_pz, raw_eog, raw_resp, raw_emg, raw_temp], axis=1)



    # Get raw header
    f = open(psg_fnames[i], 'r')
    reader_raw = dhedfreader.BaseEDFReader(f)
    reader_raw.read_header()
    h_raw = reader_raw.header
    f.close()
    raw_start_dt = datetime.strptime(h_raw['date_time'], "%Y-%m-%d %H:%M:%S")

    # Read annotation and its header
    f = open(ann_fnames[i], 'r')
    reader_ann = dhedfreader.BaseEDFReader(f)
    reader_ann.read_header()
    h_ann = reader_ann.header
    _, _, ann = zip(*reader_ann.records())
    f.close()
    ann_start_dt = datetime.strptime(h_ann['date_time'], "%Y-%m-%d %H:%M:%S")

    # Assert that raw and annotation files start at the same time
    assert raw_start_dt == ann_start_dt


    # Generate label and remove indices
    remove_idx = []    # indicies of the data that will be removed
    labels = []        # indicies of the data that have labels
    label_idx = []
    for a in ann[0]:
        onset_sec, duration_sec, ann_char = a
        ann_str = "".join(ann_char)
        label = ann2label[ann_str]
        if label != UNKNOWN:
            if duration_sec % EPOCH_SEC_SIZE != 0:
                raise Exception("Something wrong")
            duration_epoch = int(duration_sec / EPOCH_SEC_SIZE)
            label_epoch = np.ones(duration_epoch, dtype=np.int) * label
            labels.append(label_epoch)
            idx = int(onset_sec * sampling_rate) + np.arange(duration_sec * sampling_rate, dtype=np.int)
            label_idx.append(idx)

            print "Include onset:{}, duration:{}, label:{} ({})".format(
                onset_sec, duration_sec, label, ann_str
            )
        else:
            idx = int(onset_sec * sampling_rate) + np.arange(duration_sec * sampling_rate, dtype=np.int)
            remove_idx.append(idx)

            print "Remove onset:{}, duration:{}, label:{} ({})".format(
                onset_sec, duration_sec, label, ann_str
            )
    labels = np.hstack(labels)

    print "before remove unwanted: {}".format(np.arange(len(raw_ch_df)).shape)
    if len(remove_idx) > 0:
        remove_idx = np.hstack(remove_idx)
        select_idx = np.setdiff1d(np.arange(len(raw_ch_df)), remove_idx)
    else:
        select_idx = np.arange(len(raw_ch_df))

    print "after remove unwanted: {}".format(select_idx.shape)

    # Select only the data with labels
    print "before intersect label: {}".format(select_idx.shape)
    label_idx = np.hstack(label_idx)
    select_idx = np.intersect1d(select_idx, label_idx)
    print "after intersect label: {}".format(select_idx.shape)

    # Remove extra index
    if len(label_idx) > len(select_idx):
        print "before remove extra labels: {}, {}".format(select_idx.shape, labels.shape)
        extra_idx = np.setdiff1d(label_idx, select_idx)
        # Trim the tail
        if np.all(extra_idx > select_idx[-1]):
            print('debug')
            print('len(select_idx',len(select_idx),' EPOCH_SEC_SIZE', EPOCH_SEC_SIZE, 'sampling_rate', sampling_rate)
            n_trims = len(select_idx) % int(EPOCH_SEC_SIZE * sampling_rate)
            print('n_trims', n_trims)
            n_label_trims = int(math.ceil(n_trims / (EPOCH_SEC_SIZE * sampling_rate)))
            print('select_idx', select_idx, 'select_idx new', select_idx[:-n_trims])
            print('n_trims', n_trims)
            select_idx = select_idx[:-n_trims]
            print('Final', select_idx)
            labels = labels[:-n_label_trims]
        print "after remove extra labels: {}, {}".format(select_idx.shape, labels.shape)

    
    # Remove movement and unknown stages if any
    raw_ch = raw_ch_df.values[select_idx]
    print('raw_ch', len(raw_ch))
    # Verify that we can split into 30-s epochs
    if len(raw_ch) % (EPOCH_SEC_SIZE * sampling_rate) != 0:
        raise Exception("Something wrong")
    n_epochs = len(raw_ch) / (EPOCH_SEC_SIZE * sampling_rate)

    print('_____________________________________________')
    print('debugging:')
    print('len(raw.ch)', raw_ch, ' and n_epochs',n_epochs, 'EPOCH_SEC_SIZE', EPOCH_SEC_SIZE)
    print('_____________________________________________')
    # Get epochs and their corresponding labels
    x = np.asarray(np.split(raw_ch, n_epochs)).astype(np.float32)
    y = labels.astype(np.int32)
    assert len(x) == len(y)

    # Select on sleep periods
    w_edge_mins = 30
    nw_idx = np.where(y != stage_dict["W"])[0]
    start_idx = nw_idx[0] - (w_edge_mins * 2)
    end_idx = nw_idx[-1] + (w_edge_mins * 2)
    if start_idx < 0: start_idx = 0
    if end_idx >= len(y): end_idx = len(y) - 1
    select_idx = np.arange(start_idx, end_idx+1)
    print("Data before selection: {}, {}".format(x.shape, y.shape))
    x = x[select_idx]
    y = y[select_idx]
    print("Data after selection: {}, {}".format(x.shape, y.shape))

    # Save
    filename = ntpath.basename(psg_fnames[i]).replace("-PSG.edf", ".npz")
    save_dict = {
        "x": x, 
        "y": y, 
        "fs": sampling_rate,
        "ch_label": select_ch,
        "header_raw": h_raw,
        "header_annotation": h_ann,
    }
    
    resulting_name = output_dir+filename
    np.savez(resulting_name, **save_dict)

    print "\n=======================================\n"

Extracting edf Parameters from data/SC4001E0-PSG.edf...
EDF file detected
Setting channel info structure...
Created Raw.info structure...
Ready.
Extracting edf Parameters from data/SC4001E0-PSG.edf...
EDF file detected
Setting channel info structure...
Created Raw.info structure...
Ready.
scale_time is deprecated and will be removed in 0.16, use scaling_time instead


<ipython-input-15-0dbaedaa8d64>:23: DeprecationWarning: scale_time is deprecated and will be removed in 0.16, use scaling_time instead
  raw_eeg_fpz = raw.to_data_frame(scale_time=100.0)['EEG Fpz-Cz']


Converting time column to int64...
-------------------------------------------------------------------
scale_time is deprecated and will be removed in 0.16, use scaling_time instead


<ipython-input-15-0dbaedaa8d64>:27: DeprecationWarning: scale_time is deprecated and will be removed in 0.16, use scaling_time instead
  raw_eeg_pz = raw.to_data_frame(scale_time=100.0)['EEG Pz-Oz']


Converting time column to int64...
-------------------------------------------------------------------
scale_time is deprecated and will be removed in 0.16, use scaling_time instead


...

<ipython-input-15-0dbaedaa8d64>:42: DeprecationWarning: scale_time is deprecated and will be removed in 0.16, use scaling_time instead
  raw_emg = raw.to_data_frame(scale_time=100.0)['EMG submental']


Converting time column to int64...
-------------------------------------------------------------------
scale_time is deprecated and will be removed in 0.16, use scaling_time instead


<ipython-input-15-0dbaedaa8d64>:47: DeprecationWarning: scale_time is deprecated and will be removed in 0.16, use scaling_time instead
  raw_temp = raw.to_data_frame(scale_time=100.0)['Temp rectal']


Converting time column to int64...
-------------------------------------------------------------------
Include onset:0.0, duration:39300.0, label:0 (Sleep stage W)
Include onset:39300.0, duration:330.0, label:1 (Sleep stage 1)
Include onset:39630.0, duration:450.0, label:0 (Sleep stage W)
Include onset:40080.0, duration:60.0, label:1 (Sleep stage 1)
Include onset:40140.0, duration:90.0, label:0 (Sleep stage W)
Include onset:40230.0, duration:120.0, label:1 (Sleep stage 1)
Include onset:40350.0, duration:180.0, label:0 (Sleep stage W)
Include onset:40530.0, duration:30.0, label:1 (Sleep stage 1)
Include onset:40560.0, duration:60.0, label:0 (Sleep stage W)
Include onset:40620.0, duration:30.0, label:1 (Sleep stage 1)
Include onset:40650.0, duration:1620.0, label:0 (Sleep stage W)
Include onset:42270.0, duration:90.0, label:1 (Sleep stage 1)
Include onset:42360.0, duration:420.0, label:0 (Sleep stage W)
Include onset:42780.0, duration:90.0, label:1 (Sleep stage 1)
Include onset:42870.0, 

after intersect label: (7815000,)
('raw_ch', 7815000)
_____________________________________________
debugging:
('len(raw.ch)', array([[-4.90769231e+00, -3.03076923e+00,  4.72796093e+00,
         9.70000000e+07,  2.67200000e+00,  3.69976854e+07],
       [-1.26051282e+01,  8.22637363e+00,  4.23028083e+00,
         9.70000000e+07,  2.67000000e+00,  3.69946301e+07],
       [ 1.81538462e+00,  7.21611722e-01,  5.72332112e+00,
         9.70000000e+07,  2.66800000e+00,  3.69925933e+07],
       ...,
       [-5.27487179e+01, -2.45347985e+00, -8.08730159e+01,
         5.30000000e+07,  3.71400000e+00,  3.53172854e+07],
       [-1.51384615e+01,  1.68376068e+00, -8.43567766e+01,
         5.30000000e+07,  3.71600000e+00,  3.53172854e+07],
       [-3.38461538e+01,  1.97240537e+00, -7.24124542e+01,
         5.30000000e+07,  3.71800000e+00,  3.53172854e+07]]), ' and n_epochs', 2605.0, 'EPOCH_SEC_SIZE', 30)
_____________________________________________
Data before selection: (2605, 3000, 6), (2605,)
Data

In [17]:
h_ann

{'EDF+': True,
 'contiguous': True,
 'date_time': '2089-07-20 15:55:00',
 'digital_max': array([32767.]),
 'digital_min': array([-32768.]),
 'label': ['EDF Annotations'],
 'local_recording_id': 'Startdate 20-JUL-1989 X X X',
 'local_subject_id': 'X M X Male_28yr',
 'n_channels': 1,
 'n_records': 1,
 'n_samples_per_record': [1717],
 'physical_max': array([1.]),
 'physical_min': array([0.]),
 'prefiltering': [''],
 'record_length': 0.0,
 'transducer_type': [''],
 'units': ['']}

In [26]:
import numpy as np
import pandas as pd

data_train = np.load('../Alexanders_Sleep/data/output/SC4001E0.npz')
'x shape is ',data_train.f.x.shape

('x shape is ', (841, 3000, 6))

In [7]:
data_train = data_train[:, :, np.newaxis, np.newaxis]
data_train

TypeError: not all arguments converted during string formatting

In [25]:
data_train = np.load('data/output/SC4001E0.npz')
'x shape is ',data_train.f.x.shape

('x shape is ', (841, 3000, 6))